In [ ]:
interpreter = tf.lite.Interpreter(model_path='model.tflite')
interpreter.allocate_tensors()


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        'Dataset_3Devices/test',
        target_size=target_image_size,
        batch_size=32,
        class_mode='categorical',
        shuffle=False
)


In [ ]:
predictions = []
for x_batch, _ in test_generator:
    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], x_batch)
    interpreter.invoke()
    output_tensor = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])
    predictions.extend(output_tensor)


In [ ]:
predicted_classes = np.argmax(predictions, axis=1)
class_names = list(test_generator.class_indices.keys())
